In [7]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
%matplotlib inline

In [8]:
df_lists = sorted(glob.glob('../data/*.csv'))
print(f'all files are {len(df_lists)}')
df = None
for df_path in df_lists:
    if df is None:
        df = pd.read_csv(df_path)
    else:
        tmp = pd.read_csv(df_path)
        df = pd.concat([df,tmp])
df = df.reset_index(drop=True)


all files are 156


In [3]:
df.to_csv('../full_list.csv',index=None)

In [10]:
df[' protocol_name'].head(20)#.fillna('nan').str.lower().str.contains('|'.join(searchfor))

0     6.9 ThorAbd Liver Dynamic 3phase+Thorax
1     6.9 ThorAbd Liver Dynamic 3phase+Thorax
2     6.9 ThorAbd Liver Dynamic 3phase+Thorax
3     6.9 ThorAbd Liver Dynamic 3phase+Thorax
4     6.9 ThorAbd Liver Dynamic 3phase+Thorax
5     6.9 ThorAbd Liver Dynamic 3phase+Thorax
6     6.9 ThorAbd Liver Dynamic 3phase+Thorax
7     6.9 ThorAbd Liver Dynamic 3phase+Thorax
8     6.9 ThorAbd Liver Dynamic 3phase+Thorax
9                              5.10 ThorPel P
10                             5.10 ThorPel P
11                             5.10 ThorPel P
12                                  胸部　単純　SHR
13                                  胸部　単純　SHR
14                                  胸部　単純　SHR
15          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
16          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
17          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
18          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
19          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
Name:  protocol_name, dtype: object

In [12]:
new_df = pd.DataFrame([])

df['ID'] = df['Facility_Code']+df[' Accesion_Number']
ids = df['ID'].unique()
##腹部以外のprotocol を排除
searchfor = ['abd','pancreas','liver','pk','pel','renal','kub','colon','骨盤','腹','体幹']
print(df.shape)
df = df[df[' protocol_name'].fillna('nan').str.lower().str.contains('|'.join(searchfor))]
print(df.shape)

df = df[df[' image_type'].str.lower().str.contains('axial')]
print(df.shape)
###スライス少ないのはスカウト等なので排除
###window_centerがマイナスなのは肺野なので排除。それと一緒に取っている同じスライス数の画像もおそらく胸部の縦隔条件なので排除。

for id in tqdm(ids):
    tmp = df[df['ID'] == id].reset_index(drop=True)
    num_slices = tmp[' Number_of_Slices']
    window_isminus = tmp[' window_center'].str.contains('-')
    num_slices_low = num_slices<40
    lung_slices = num_slices[window_isminus]
    lung_slices = np.isin(num_slices,lung_slices)

    exclude_slices = num_slices_low | lung_slices
    

    tmp = tmp[~exclude_slices].reset_index(drop=True)

    new_df = pd.concat([new_df,tmp])

new_df = new_df.reset_index(drop=True)
new_df['path'] = new_df['Facility_Code'].astype('str')+'_'+new_df[' Accesion_Number'].astype('int').astype('str')+\
    '_'+new_df[' Series_Number'].fillna(9999).astype('int').astype('str').str.zfill(4)+'_'+new_df[' Number_of_Slices'].astype(int).astype('str').str.zfill(3)+'.nii.gz'

(59128, 18)
(59128, 18)
(59128, 18)


100%|██████████| 20917/20917 [01:27<00:00, 237.88it/s]


In [13]:
print(new_df.shape)
new_df.head()


(42824, 18)


,Facility_Code,Accesion_Number,Series_Number,Number_of_Slices,modality,slice_thickness,patient_sex,patient_age,contrast_bolus_agent,manufacturer,model_name,conv_kernel,window_center,window_width,patient_orientation,image_type,protocol_name,ID
0,43157400144,1148655720201211.0,2.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,35,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0
1,43157400144,1148655720201211.0,3.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0
2,43157400144,1148655720201211.0,4.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0
3,43157400144,1148655720201211.0,5.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0
4,43157400144,1167702520201223.0,2.0,131,CT,5.0,F,072Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,30,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",5.10 ThorPel P,1167745677601367.0


In [30]:

new_df.to_csv('../include_abdomen.csv',index=False,encoding='utf_8_sig')
new_df['path'].to_csv('../include_abdomen_path.csv',index=False,encoding='utf_8_sig')

In [25]:
new_df[' Number_of_Slices'].value_counts().sort_index().tail(60)

318       7
319       3
321       5
322       5
324       2
326       4
327       2
328       3
329       4
330       2
332       3
334       3
336       2
337       2
339       1
340       3
342       3
344       1
345       1
351       4
352       4
354       1
355       5
357       3
358       2
359       2
360       1
363       1
369       1
374       2
403       1
408       6
420      12
426       1
430       2
432       1
433       2
438       1
447       1
448       1
480       1
482       4
491       2
496       1
502       2
506       2
512       2
516       1
538       1
547       1
564       1
567       2
581       2
674       1
759       1
770       1
954       1
1098      1
5824      4
16768     4
Name:  Number_of_Slices, dtype: int64

In [31]:
df = pd.read_csv('../rename.csv')
new_df.merge(df,how='left',right_on='original_name',left_on='path')

,Facility_Code,Accesion_Number,Series_Number,Number_of_Slices,modality,slice_thickness,patient_sex,patient_age,contrast_bolus_agent,manufacturer,...,conv_kernel,window_center,window_width,patient_orientation,image_type,protocol_name,ID,path,original_name,renamed
0,43157400144,1148655720201211.0,2.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,...,STANDARD,35,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0002_053.nii.gz,43157400144_1148655720201211_0002_053.nii.gz,jmid_0024606_0000.nii.gz
1,43157400144,1148655720201211.0,3.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,...,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0003_053.nii.gz,43157400144_1148655720201211_0003_053.nii.gz,jmid_0024607_0000.nii.gz
2,43157400144,1148655720201211.0,4.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,...,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0004_053.nii.gz,43157400144_1148655720201211_0004_053.nii.gz,jmid_0024608_0000.nii.gz
3,43157400144,1148655720201211.0,5.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,...,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0005_053.nii.gz,43157400144_1148655720201211_0005_053.nii.gz,jmid_0024609_0000.nii.gz
4,43157400144,1167702520201223.0,2.0,131,CT,5.0,F,072Y,NaN,GE MEDICAL SYSTEMS,...,STANDARD,30,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",5.10 ThorPel P,1167745677601367.0,43157400144_1167702520201223_0002_131.nii.gz,43157400144_1167702520201223_0002_131.nii.gz,jmid_0024627_0000.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42819,118,6010016870769.0,201,131,CT,5.0,F,034Y,NaN,Philips,...,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Chest-Pelivis /iCT in HU,6010016870887.0,118_6010016870769_0201_131.nii.gz,118_6010016870769_0201_131.nii.gz,jmid_0000200_0000.nii.gz
42820,118,6010018198758.0,201,148,CT,5.0,F,062Y,NaN,Philips,...,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018198876.0,118_6010018198758_0201_148.nii.gz,118_6010018198758_0201_148.nii.gz,jmid_0000305_0000.nii.gz
42821,118,6010018748732.0,301,164,CT,5.0,M,076Y,NaN,Philips,...,B,"[40, 40]","[300, 300]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018748850.0,118_6010018748732_0301_164.nii.gz,118_6010018748732_0301_164.nii.gz,jmid_0000374_0000.nii.gz
42822,118,6010018776642.0,201,170,CT,5.0,M,082Y,NaN,Philips,...,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018776760.0,118_6010018776642_0201_170.nii.gz,118_6010018776642_0201_170.nii.gz,jmid_0000375_0000.nii.gz


In [46]:
sm = 0
for i in np.arange(0,450,50):
    tmp = new_df[(new_df[' Number_of_Slices']>=i) & (new_df[' Number_of_Slices']<i+50)]
    print(i,i+50,len(tmp))
    sm += len(tmp)

0 50 2741
50 100 17843
100 150 15994
150 200 5494
200 250 356
250 300 168
300 350 139
350 400 26
400 450 28


In [47]:
sm

42789

In [50]:
new_df

,Facility_Code,Accesion_Number,Series_Number,Number_of_Slices,modality,slice_thickness,patient_sex,patient_age,contrast_bolus_agent,manufacturer,model_name,conv_kernel,window_center,window_width,patient_orientation,image_type,protocol_name,ID,path
0,43157400144,1148655720201211.0,2.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,35,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0002_053.nii.gz
1,43157400144,1148655720201211.0,3.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0003_053.nii.gz
2,43157400144,1148655720201211.0,4.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0004_053.nii.gz
3,43157400144,1148655720201211.0,5.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0005_053.nii.gz
4,43157400144,1167702520201223.0,2.0,131,CT,5.0,F,072Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,30,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",5.10 ThorPel P,1167745677601367.0,43157400144_1167702520201223_0002_131.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42819,118,6010016870769.0,201,131,CT,5.0,F,034Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Chest-Pelivis /iCT in HU,6010016870887.0,118_6010016870769_0201_131.nii.gz
42820,118,6010018198758.0,201,148,CT,5.0,F,062Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018198876.0,118_6010018198758_0201_148.nii.gz
42821,118,6010018748732.0,301,164,CT,5.0,M,076Y,NaN,Philips,iCT 256,B,"[40, 40]","[300, 300]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018748850.0,118_6010018748732_0301_164.nii.gz
42822,118,6010018776642.0,201,170,CT,5.0,M,082Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018776760.0,118_6010018776642_0201_170.nii.gz


In [49]:
new_df[new_df[' Number_of_Slices']>450].shape


(35, 19)

In [ ]:
import os
import shutil

data_dir = '../data/finish_pred/'
finished_list = []
finished_paths = sorted(glob.glob(os.path.join(data_dir,'*.nii.gz')))
for path in finished_paths:
    img_path = path.split('.')[0]+'_0000.nii.gz'
    finished_list.append(img_path)

pd.DataFrame(finished_list).T.to_csv('finished.csv',index=None)
